In [1]:
cd /Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Library/Caches/pypoetry/virtualenvs/ploutos-syHIIjPM-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:

from pandas import Timestamp

from dotenv import load_dotenv
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from utils.sheet_migration import SheetMigration
load_dotenv(dotenv_path="notebook/.env", override=True)
import sys
from importlib import reload
from src.api.bank_api import BankApi, Bank
if 'api.bank_api' in sys.modules:
    reload(sys.modules['src.api.bank_api'])
else:
    print("BankApi module not loaded yet - no need to reload")


BankApi module not loaded yet - no need to reload


### Sheet Migration

In [ ]:
from ploutos.api.bank_api import BankApi, Bank
if 'db.models' in sys.modules:
    reload(sys.modules['db.models'])
else:
    print("db.models module not loaded yet - no need to reload")

from db.models import *
#account_api=BankApi(Bank.LCL)

#print(account_api.get_metadata())

db.models module not loaded yet - no need to reload


In [6]:
# sheet_migration=SheetMigration()

# print(sheet_migration.account_list)


In [ ]:
df_paiement=sheet_migration.get_data("Paiement")
df_transfert=sheet_migration.get_data("Transfert")
df_benefices=sheet_migration.get_data("Bénéfices")


In [ ]:
accounts_to_create_budget=[]
for index, row in df_paiement[['Catégorie','Sous Catégorie']].drop_duplicates().iterrows():
    accounts_to_create_budget.append(AccountCreate(
        category=row['Catégorie'] or "Unknown",
        sub_category="Unknown",
        is_real=False,
        name=row['Sous Catégorie'] or "Unknown",
        original_amount=0
    ))
print("Budget accounts to create: ",len(accounts_to_create_budget))

In [ ]:
accounts_to_creat_real=[]
for index, row in df_paiement[['Compte']].drop_duplicates().iterrows():
    accounts_to_creat_real.append(AccountCreate(
        category=row['Compte'] or "Unknown",
        sub_category="Unknown",
        is_real=True,
        name=row['Compte'] or "Unknown",
        original_amount=0
    ))
print("Real accounts to create: ",len(accounts_to_creat_real))


In [ ]:
accounts_to_create_missing=[]
for account in set(df_transfert['Compte départ']).union(set(df_transfert['Compte arrivée'])).union(set(df_benefices['Compte'])) -set(df_paiement['Compte']):
    accounts_to_create_missing.append(AccountCreate(
        category=account or "Unknown",
        sub_category="Unknown",
        is_real=True,
        name=account or "Unknown",
        original_amount=0
    ))
accounts_to_create_missing.append(AccountCreate(
        category="Unknown",
        sub_category="Unknown",
        is_real=True,
        name="Unknown",
        original_amount=0
    ))
print("Transfer and benefit accounts to create",len(accounts_to_create_missing))



In [8]:
from api.deps import get_db
from api.routers.accounts import *
import importlib
from api.routers.accounts import create_account
importlib.reload(importlib.import_module('api.routers.accounts'))


from fastapi import Depends
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Create accounts from sheet data
async def create_accounts_from_sheet(accounts_to_create = []):
    db = get_db
    
    # Define account types based on sheet data
    
    # Create accounts
    for account in accounts_to_create:
        try:
            await create_account(account, db)
            print(f"Created account: {account.name}")
        except Exception as e:
            print(f"Error creating account {account.name}: {str(e)}")

# Run the account creation using asyncio.run() since we're using nest_asyncio
asyncio.run(create_accounts_from_sheet(accounts_to_creat_real+accounts_to_create_budget+accounts_to_create_missing))


NameError: name 'accounts_to_creat_real' is not defined

In [7]:
from api.routers.accounts import get_accounts
from api.deps import get_db

async def query_all_accounts():
    db = get_db
    accounts = await get_accounts(db)
    return accounts

# Run the query using asyncio.run() since we're using nest_asyncio
accounts = asyncio.run(query_all_accounts())
print("Found accounts:", len(accounts))
account_text=[accounts['name'] for accounts in accounts]
for account in accounts:
    account=Account(**account)
    print(f"- {account.name} (ID: {account.accountId})")


NameError: name 'asyncio' is not defined

In [ ]:
db = get_db
accounts = (
    db.table("Accounts")
    .select("*")
    .execute()
)
accounts = pd.DataFrame(accounts.data)


In [ ]:
from tqdm import tqdm
df_paiement['account_credit'] = None
df_paiement['account_debit'] = None

for index, row in tqdm(df_paiement.iterrows(), total=len(df_paiement)):
    try:
        debit_account = accounts[accounts['name'] == (row["Compte"] if pd.notna(row["Compte"]) else "Unknown")]
        if not debit_account.empty:
            df_paiement.at[index, 'account_debit'] = debit_account.iloc[0]['accountId']
        else:
            print(f"Warning: No matching debit account found for {row['Compte']}")
            
        credit_account = accounts[
            (accounts['name'] == (row["Sous Catégorie"] if row["Sous Catégorie"] else "Unknown")) &
            (accounts['category'] == (row["Catégorie"] if row["Catégorie"] else "Unknown")) & (accounts['is_real']==False)
        ]
        if not credit_account.empty:
            df_paiement.at[index, 'account_credit'] = credit_account.iloc[0]['accountId']
        else:
            print(f"Warning: No matching credit account found for {index}")
    except Exception as e:
        raise e
        print(f"Error getting account IDs for row {index}: {str(e)}")

In [ ]:
print(df_paiement[df_paiement["account_credit"].isna()])
print(df_paiement[df_paiement["account_debit"].isna()])



In [ ]:
import importlib
import db.models
importlib.reload(db.models)

from db.models import TransactionCreate, TransactionSlaveCreate
from db.migrations import upload_transactions

In [ ]:
from uuid import UUID, uuid4
from datetime import datetime
master_transactions = []
slaves_transactions = []
for index, row in df_paiement.iterrows():
    # Get account type ID for this transaction
    account_credit = row["account_credit"]
    account_debit = row["account_debit"]
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=debitId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=creditId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_credit,
            masterId=debitId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")



In [ ]:
df_benefices=df_benefices.merge(accounts,how='left',left_on='Compte',right_on='name')
df_transfert=df_transfert.merge(accounts[['accountId','name']].rename(columns={'accountId':'departID'}), how='left', left_on='Compte départ', right_on='name').merge(accounts[['accountId','name']].rename(columns={'accountId':'arriveeID'}), how='left', left_on='Compte arrivée', right_on='name')

In [ ]:
unknown_id = accounts[(accounts['sub_category'] == 'Unknown') & (accounts['category'] == 'Unknown') & (accounts['is_real'] == False) & (accounts['name'] == 'Unknown')]['accountId'].iloc[0]
unknown_id

In [ ]:
for index, row in df_benefices.iterrows():
    # Get account type ID for this transaction
    account_credit = row["accountId"]
    account_debit = unknown_id
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=creditId,
            created_at=date,
            updated_at=date,
            description=row["Raison"],
            date=pd.Timestamp(row["Date"]),
            type="credit",
            amount=row["Montant"],
            accountId=account_credit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=debitId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_debit,
            masterId=creditId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")

In [ ]:
for index, row in df_transfert.iterrows():
    # Get account type ID for this transaction
    account_credit = row["arriveeID"]
    account_debit = row['departID']
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=debitId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=creditId,
            created_at=date,
            updated_at=date,
            type="credit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_credit,
            masterId=debitId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")

In [ ]:
assert len(master_transactions)==1581

In [ ]:

        

# Call the function with the database session and lists
upload_transactions(get_db, master_transactions, slaves_transactions)


In [ ]:
accounts

In [ ]:
from uuid import UUID, uuid4
from datetime import datetime
master_transactions = []
slaves_transactions = []
for index, row in df_paiement.iterrows():
    # Get account type ID for this transaction
    account_credit = row["account_credit"]
    account_debit = row["account_debit"]
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=creditId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=-row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=debitId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_debit,
            masterId=creditId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")
upload_transactions(get_db, master_transactions, slaves_transactions)


### LCL

#### Compte Courant

In [ ]:
from ploutos.utils.scraping.scrap_lcl import convert_LCLpdf_to_df

old_lcl=convert_LCLpdf_to_df("ploutos/data/lcl/compte_courant")

In [ ]:
def calculate_last_transaction(db_payement, db_transfert, db_benefices,account):
    last_transaction=[]
    last_transaction.append(db_payement[db_payement["Compte"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte arrivée"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte départ"]==account]["Date"].max())
    last_transaction.append(db_benefices[db_benefices["Compte"]==account]["Date"].max())
    last_transaction=[ts for ts in last_transaction if pd.notna(ts)]
    if len(last_transaction)>0:
        last_transaction=max(last_transaction)
    else:
        last_transaction=None
    return last_transaction


In [ ]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Compte Courant").strftime("%Y-%m-%d")
transactions=account_api.get_transactions(date_from='2024-06-06')
print(last_transaction)


In [ ]:
debit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Compte Courant"]["Actuel"].values[0].replace("€","").replace(",","."))
end_september_balance=float(sheet_value)+credit_old-debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance+transactions[transactions["Date"]>'2024-09-30']['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=account_api.get_balance()
print(f"Official balance: {official_balance}")


#### Compte Epargne


Le livret jeune n'a pas subi de changement et va fermer. On regarde uniquement le livret A.
On détecte les transactions de M JEREMY BON dans le compte courant avec `(transactions["Description"].str.contains("M JEREMY BON"))`

In [ ]:
df_transfert[df_transfert["Compte départ"]=="Livret A"]

In [ ]:
convert_LCLpdf_to_df("ploutos/data/lcl/compte_livret")
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Livret A")
print(last_transaction)


In [ ]:
debit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Livret A"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
end_september_balance=float(sheet_value)-credit_old+debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance-transactions[(transactions["Description"].str.contains("M JEREMY BON"))&(transactions["Date"]>'2024-09-30')]['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=8000
print(f"Official balance: {official_balance}")

### Lydia

In [ ]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Lydia")
print(last_transaction)
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Lydia"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
print(f"Lydia balance on {last_transaction.strftime('%Y-%m-%d')}: {sheet_value}")


In [ ]:
lydia_account_value=50.84
new_transaction_lydia=pd.DataFrame([{
    "Date":"2024-09-01",
    "Libellé":"Madrid Gael Terier",
    "Montant":38.89,
    "Compte":"Lydia",
},
{
    "Date":"2024-08-31",
    "Libellé":"Guilhem Leborgne",
    "Montant": -4,
    "Compte":"Lydia",
    
},
{
    "Date":"2024-08-27",
    "Libellé":"Dépôt crte revolut 7255",
    "Montant": 25,
    "Compte":"Lydia",
},{
    "Date":"2024-08-27",
    "Libellé":"Guilhem Leborgne",
    "Montant": -25,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Kdo Juan Lucie Formery",
    "Montant": -8.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Juanita PA",
    "Montant": -3.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-16",
    "Libellé":"Tren Gijon Lucie Formery",
    "Montant": 27.1,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-01",
    "Libellé":"Thai Bastien Englebert",
    "Montant": -7.9,
    "Compte":"Lydia",
}
])
assert new_transaction_lydia['Montant'].sum()+sheet_value==lydia_account_value



### Revolut


In [ ]:
cd c:\Users\bonje\Documents\Perso\ploutos


In [ ]:

from ploutos.api.bank_api import BankApi, Bank


In [ ]:
lcl_api.get_transactions()

In [ ]:
lcl_api=BankApi(Bank.LCL)
lcl_api.api.get_transactions()


In [ ]:
revolut_api=BankApi(Bank.REVOLUT)


In [ ]:
raw_transac=revolut_api.api.get_transactions()

In [ ]:
pd.DataFrame(raw_transac['transactions']['pending'])

In [ ]:
transac_rev=revolut_api.get_transactions()


### Autres

In [ ]:
transac_rev

In [ ]:
sheet_migration.account_list

    ### Tricount (A faire)

### ticket resto (A solde)

#### Natixis Interépargne 


### Retrieve previous transactions

In [1]:
cd /Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Library/Caches/pypoetry/virtualenvs/ploutos-syHIIjPM-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from api.deps import get_db
from api.routers.accounts import *
import importlib
import pandas as pd
importlib.reload(importlib.import_module('api.bank_api'))
from api.routers.accounts import get_current_amounts
from fastapi import Depends
import nest_asyncio
from api.bank_api import BankApi
import warnings
from uuid import UUID, uuid4
warnings.filterwarnings("ignore")


In [3]:
async def query_all_accounts():
    db = get_db
    accounts = await get_accounts(db)
    return accounts

In [4]:
all_accounts=pd.DataFrame(await query_all_accounts())

In [5]:
from api.routers.accounts import get_current_amounts

async def query_all_real_accounts():
    accounts = await get_current_amounts(get_db)
    return accounts

# If running in Jupyter, use nest_asyncio and run the coroutine directly
real_accounts = await query_all_real_accounts()
df_accounts = pd.DataFrame([a.model_dump() for a in real_accounts])
unknown_id = all_accounts[(all_accounts['sub_category'] == 'Unknown') & (all_accounts['category'] == 'Unknown') & (all_accounts['is_real'] == False) & (all_accounts['name'] == 'Unknown')]['accountId'].iloc[0]
df_accounts.head(2)

,account_id,name,category,sub_category,current_amount,is_real,max_date
0,42aa4dec-ae44-4898-a0f5-17b063283314,Unknown,Unknown,Unknown,0.000000e+00,True,NaT
1,e775ea39-f089-4432-83c0-f2477aba9248,Lydia,Lydia,Unknown,2.999379e-12,True,2025-06-23 13:10:44.393768+00:00


In [6]:
account_id,current_amount,max_date=df_accounts[df_accounts['name'].str.contains("LCL")][['account_id','current_amount','max_date']].iloc[0]

In [7]:
unknown_id

'27d40c76-c20b-4930-b640-c590bd0e3b7b'

### Old LCL

In [54]:
from utils.scraping.scrap_lcl import convert_LCLpdf_to_df
from datetime import datetime
max_date_lcl = max_date.strftime("%Y-%m-%d")
max_date=pd.Timestamp("2025-07-31")
if account_id=="5e139075-c4b5-4f6f-afdb-e8a5efa9d377":
    old_lcl=convert_LCLpdf_to_df("data/lcl/compte_courant")
    old_lcl=old_lcl[old_lcl["Date"]>max_date_lcl]

100%|██████████| 2/2 [00:02<00:00,  1.11s/it]
2it [00:00, 4937.38it/s]
/Users/jeremy_bon/Documents/Perso/Ploutos/ploutos/utils/scraping/scrap_lcl.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["VALEUR"], dayfirst=True)


In [89]:
old_balance=old_lcl["CREDIT"].sum()-old_lcl["DEBIT"].sum()+current_amount
try:
    assert abs(round(old_balance,2) - float(2805.4))==0
    print("Balance matches expected value.")
except AssertionError:
    print(f"Balance mismatch: calculated {round(old_balance,2) }, expected 2805.4")

Balance matches expected value.


In [114]:
old_lcl["Montant"]=old_lcl["CREDIT"]+old_lcl["DEBIT"]
old_lcl["type"]=old_lcl["DEBIT"].apply(lambda x: "credit" if x==0 else "debit")
old_lcl

,Date,Description,VALEUR,DEBIT,CREDIT,Montant,type
19,2025-06-21,VIR INST Jeremy Bon revo,21.06.25,250.00,0.00,250.00,debit
20,2025-06-21,VIREMENT M JEREMY BON,21.06.25,0.00,250.00,250.00,credit
21,2025-06-25,VIREMENT M JEREMY BON,25.06.25,0.00,300.00,300.00,credit
22,2025-06-25,VIR INST Jeremy Bon revo,25.06.25,300.00,0.00,300.00,debit
23,2025-06-30,VIREMENT CARREFOUR FRANCE,30.06.25,0.00,3417.71,3417.71,credit
24,2025-06-30,VIR INST vulpian nouveau,30.06.25,668.45,0.00,668.45,debit
25,2025-06-30,VIR SEPA M JEREMY BON,30.06.25,550.00,0.00,550.00,debit
26,2025-07-02,VIR INST PAPEREUX VERONIQUE,02.07.25,0.00,858.89,858.89,credit
27,2025-07-02,VIR INST PAPEREUX VERONIQUE,02.07.25,0.00,9.99,9.99,credit
28,2025-07-03,VIR INST Jeremy Bon revo,03.07.25,1000.00,0.00,1000.00,debit


In [118]:
from uuid import UUID, uuid4
from datetime import datetime
from db.models import Transaction, TransactionSlave
from db.migrations import upload_transactions
master_transactions = []
slaves_transactions = []
for index, row in old_lcl.iterrows():
    # Get account type ID for this transaction
    real_account = account_id
    slave_account = unknown_id
    date=pd.Timestamp(datetime.now())
    masterId=uuid4()
    slaveId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=masterId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type=row["type"],
            amount=row["Montant"],
            accountId=real_account
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=slaveId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=slave_account,
            masterId=masterId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")
upload_transactions(get_db, master_transactions, slaves_transactions)


100%|██████████| 32/32 [00:05<00:00,  5.45it/s]


In [120]:
pd.Timestamp(row["Date"])

Timestamp('2025-07-31 00:00:00')

### All accounts go cardless

LCL, Fortuneo, Revolut

In [10]:
account_id,current_amount,max_date=df_accounts[df_accounts['name'].str.contains("LCL")][['account_id','current_amount','max_date']].iloc[0]

In [11]:
account_api=BankApi(accountId=account_id)

Successfully connected to LCL_CRLYFRPP


In [12]:
account_api.balances

In [27]:
balances=account_api.get_balances()
transactions=account_api.get_transactions()

[{'balanceAmount': {'amount': '424.66', 'currency': 'EUR'}, 'balanceType': 'expected', 'referenceDate': '2025-11-07'}, {'balanceAmount': {'amount': '392.35', 'currency': 'EUR'}, 'balanceType': 'closingBooked', 'referenceDate': '2025-11-06'}]


In [31]:
official_balance=balances
print(f"Official balance: {official_balance}")

Official balance: 392.35


In [32]:
transactions["type"]= transactions["Montant"].apply(lambda x: "credit" if x>0 else "debit")
dict_rename={
    "Description":"description",
}
if  "masterId" not in transactions.columns and   "transactionId" in transactions.columns:
    dict_rename["internalTransactionId"]="masterId"
elif "masterId" not in transactions.columns and "internalTransactionId"  in transactions.columns:
    dict_rename["transactionId"]="masterId"
transactions.rename(columns=dict_rename,inplace=True) 
transactions["amount"]=abs(transactions["Montant"])
transactions['real_account']=account_id
transactions['slave_account']=unknown_id
transactions['Date']=pd.to_datetime(transactions['Date valeur'])
transactions['slaveId']=transactions['masterId'].apply(lambda x: uuid4())

In [33]:
transactions['Montant'].sum()

np.float64(-1213.05)

In [34]:
balances

'392.35'

In [35]:
max_date

Timestamp('2025-10-17 00:00:00+0000', tz='UTC')

In [36]:
transactions[transactions["Date"].dt.date > max_date.date()- pd.Timedelta(days=1)]

,transactionId,entryReference,Date,Date valeur,Montant_dict,description,masterId,Montant,Type,type,amount,real_account,slave_account,slaveId
0,f09f67b6-f9ab-4d70-84b3-db292af1c78c,f09f67b6-f9ab-4d70-84b3-db292af1c78c,2025-11-05,2025-11-05,"{'amount': '-250', 'currency': 'EUR'}",VIREMENT PERMANENT\n\nVIR.PERMANENT Fortuneo J...,5510ced7f316cd006b6b632ab762ba03,-250.00,Booked,debit,250.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,3d8f532d-437c-4962-9ba3-728a6c920a7c
1,5e9514b7-e3b2-4861-974d-dab951569c0c,5e9514b7-e3b2-4861-974d-dab951569c0c,2025-11-05,2025-11-05,"{'amount': '-500', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA EPA...,904949c268de2365f041d4ec3b95b2d4,-500.00,Booked,debit,500.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,53de34bd-7ef4-4a9e-9999-5a6662fcbd4a
2,3729fb35-2581-4433-be98-cb51a13b63d3,3729fb35-2581-4433-be98-cb51a13b63d3,2025-11-05,2025-11-05,"{'amount': '49', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Wero M JEAN BE...,2f4986af3b852075ef2a6259ef8d1121,49.00,Booked,credit,49.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,b3e12cc5-ad3d-46f2-bc28-ba7df5a26f86
3,089445d2-05d5-40c8-9de7-2ece699558b3,089445d2-05d5-40c8-9de7-2ece699558b3,2025-11-04,2025-11-04,"{'amount': '75', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST PAPEREUX VERON...,8a8244b10e888e0ad038efba57a2a7dc,75.00,Booked,credit,75.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,9d0d7dd5-9d73-42ce-a365-6f6058f5b491
4,70f4927f-954f-4eca-bd92-989750910324,70f4927f-954f-4eca-bd92-989750910324,2025-11-03,2025-11-03,"{'amount': '-1200', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Jeremy Bon rev...,8a8f0d94c420d160340b8e45fd510c43,-1200.00,Booked,debit,1200.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,1a8ee2c6-4b50-423c-9c0b-506ab30cae61
5,27dc3fd0-dd4c-4b53-bf01-584fc316a146,27dc3fd0-dd4c-4b53-bf01-584fc316a146,2025-11-03,2025-11-03,"{'amount': '500', 'currency': 'EUR'}",VIREMENT\n\nVIREMENT M JEREMY BON\n\n\n\nSCR53...,57dd15a180107313d0488fe5728690ea,500.00,Booked,credit,500.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,f3febde2-102f-42d5-9eac-554a13181ef8
6,7855ab9c-6171-4aab-920f-62d88940a230,7855ab9c-6171-4aab-920f-62d88940a230,2025-10-31,2025-10-31,"{'amount': '1510.52', 'currency': 'EUR'}",VIREMENT SEPA RECU\n\nVIREMENT CARREFOUR FRANC...,0f494d3e35cf690805017876ed1da0f8,1510.52,Booked,credit,1510.52,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,195df18c-1427-47e8-a7af-3dcf4af934c4
7,f2961d81-c2d2-43d4-be54-6494e93823cc,f2961d81-c2d2-43d4-be54-6494e93823cc,2025-10-29,2025-10-29,"{'amount': '858.89', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST PAPEREUX VERON...,296de55484b523e63413786e674a0f3e,858.89,Booked,credit,858.89,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,e0266e49-741f-4e1d-b5bc-a4634038bd98
8,3593cc28-aa43-43b7-a99c-afe9cff17474,3593cc28-aa43-43b7-a99c-afe9cff17474,2025-10-28,2025-10-28,"{'amount': '-8.5', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CAR...,8049d60b5bef9f80c65709667f8ea77b,-8.50,Booked,debit,8.50,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,5e007d5f-34d4-41f6-aa78-761f6c29f707
9,be886669-cc81-4a8e-822c-c5690bad927d,be886669-cc81-4a8e-822c-c5690bad927d,2025-10-27,2025-10-27,"{'amount': '-2.58', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA CAR...,2f2cfab405c559093806913fab374077,-2.58,Booked,debit,2.58,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,40b5148c-c770-4198-ae04-14118f6922e6


In [37]:
transactions

,transactionId,entryReference,Date,Date valeur,Montant_dict,description,masterId,Montant,Type,type,amount,real_account,slave_account,slaveId
0,f09f67b6-f9ab-4d70-84b3-db292af1c78c,f09f67b6-f9ab-4d70-84b3-db292af1c78c,2025-11-05,2025-11-05,"{'amount': '-250', 'currency': 'EUR'}",VIREMENT PERMANENT\n\nVIR.PERMANENT Fortuneo J...,5510ced7f316cd006b6b632ab762ba03,-250.00,Booked,debit,250.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,3d8f532d-437c-4962-9ba3-728a6c920a7c
1,5e9514b7-e3b2-4861-974d-dab951569c0c,5e9514b7-e3b2-4861-974d-dab951569c0c,2025-11-05,2025-11-05,"{'amount': '-500', 'currency': 'EUR'}",PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA EPA...,904949c268de2365f041d4ec3b95b2d4,-500.00,Booked,debit,500.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,53de34bd-7ef4-4a9e-9999-5a6662fcbd4a
2,3729fb35-2581-4433-be98-cb51a13b63d3,3729fb35-2581-4433-be98-cb51a13b63d3,2025-11-05,2025-11-05,"{'amount': '49', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Wero M JEAN BE...,2f4986af3b852075ef2a6259ef8d1121,49.00,Booked,credit,49.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,b3e12cc5-ad3d-46f2-bc28-ba7df5a26f86
3,089445d2-05d5-40c8-9de7-2ece699558b3,089445d2-05d5-40c8-9de7-2ece699558b3,2025-11-04,2025-11-04,"{'amount': '75', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST PAPEREUX VERON...,8a8244b10e888e0ad038efba57a2a7dc,75.00,Booked,credit,75.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,9d0d7dd5-9d73-42ce-a365-6f6058f5b491
4,70f4927f-954f-4eca-bd92-989750910324,70f4927f-954f-4eca-bd92-989750910324,2025-11-03,2025-11-03,"{'amount': '-1200', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Jeremy Bon rev...,8a8f0d94c420d160340b8e45fd510c43,-1200.00,Booked,debit,1200.00,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,1a8ee2c6-4b50-423c-9c0b-506ab30cae61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,d0fc1e0f-b3dd-4e0f-a2a7-47c268d745c2,d0fc1e0f-b3dd-4e0f-a2a7-47c268d745c2,2025-08-11,2025-08-11,"{'amount': '11.04', 'currency': 'EUR'}",VIREMENT SEPA RECU\n\nVIREMENT APGIS\nVirement...,5fd11969f0fbe4f158b87d4d71c6dd5e,11.04,Booked,credit,11.04,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,3dcd2c6d-0a6e-4107-9607-3a89f6ca41d7
61,c4e9af1d-6755-406f-bd5c-7f18045f5519,c4e9af1d-6755-406f-bd5c-7f18045f5519,2025-08-11,2025-08-11,"{'amount': '-55.57', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Wero HAMZA E\n...,895f0ebdb8f8fec307ca55692a874965,-55.57,Booked,debit,55.57,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,2306878c-ee13-4edd-917c-d0674e33f4cd
62,0ccebc4b-f82b-4b2a-8733-0ab2b35ed270,0ccebc4b-f82b-4b2a-8733-0ab2b35ed270,2025-08-11,2025-08-11,"{'amount': '-2.25', 'currency': 'EUR'}",DEBIT DIVERS\n\nASSURANCE MOYEN DE PAIEMENT\n\...,07204eca8380f0e24125d20e3a8d9403,-2.25,Booked,debit,2.25,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,cd1dd300-615e-4625-aaff-d97d2b53f05e
63,e61de16f-e25b-408f-972e-f3fd369a8997,e61de16f-e25b-408f-972e-f3fd369a8997,2025-08-10,2025-08-10,"{'amount': '-33.03', 'currency': 'EUR'}",VIREMENT INSTANTANE\n\nVIR INST Wero ALEXANDRE...,80753eef2ac5fec6c3085afa0b0cbf4d,-33.03,Booked,debit,33.03,5e139075-c4b5-4f6f-afdb-e8a5efa9d377,27d40c76-c20b-4930-b640-c590bd0e3b7b,0c216760-cf75-4165-83a6-ebc89cd1e9d7


In [38]:
non_numeriques_montant = transactions[~transactions["Montant"].astype(str).str.replace(',', '.').str.match(r'^-?\d+(\.\d+)?$', na=False)]

# Filtrer les valeurs non numériques dans la colonne "Montant_dict"
non_numeriques_montant_dict = transactions[~transactions["Montant_dict"].astype(str).str.replace(',', '.').str.match(r'^-?\d+(\.\d+)?$', na=False)]

print("Valeurs non numériques dans Montant :")
print(non_numeriques_montant[["Montant", "description"]])

print("\nValeurs non numériques dans Montant_dict :")
print(non_numeriques_montant_dict[["Montant_dict", "description"]])

Valeurs non numériques dans Montant :
Empty DataFrame
Columns: [Montant, description]
Index: []

Valeurs non numériques dans Montant_dict :
                               Montant_dict  \
0     {'amount': '-250', 'currency': 'EUR'}   
1     {'amount': '-500', 'currency': 'EUR'}   
2       {'amount': '49', 'currency': 'EUR'}   
3       {'amount': '75', 'currency': 'EUR'}   
4    {'amount': '-1200', 'currency': 'EUR'}   
..                                      ...   
60   {'amount': '11.04', 'currency': 'EUR'}   
61  {'amount': '-55.57', 'currency': 'EUR'}   
62   {'amount': '-2.25', 'currency': 'EUR'}   
63  {'amount': '-33.03', 'currency': 'EUR'}   
64  {'amount': '109.49', 'currency': 'EUR'}   

                                          description  
0   VIREMENT PERMANENT\n\nVIR.PERMANENT Fortuneo J...  
1   PRELVT SEPA RECU D/O CONFRERE\n\nPRLV SEPA EPA...  
2   VIREMENT INSTANTANE\n\nVIR INST Wero M JEAN BE...  
3   VIREMENT INSTANTANE\n\nVIR INST PAPEREUX VERON...  
4   VIREMENT IN

In [39]:
transactions = transactions[transactions["Date"].dt.date > max_date.date()- pd.Timedelta(days=1)]
result = round(transactions["Montant"].sum() + current_amount,2)
try:
    assert float(result) == float(official_balance)
    print("Balance matches official balance.")
except AssertionError:
    print(f"Balance mismatch: calculated {result}, official {official_balance}, difference {round(result - float(official_balance),2)}")

Balance matches official balance.


In [40]:
import importlib
importlib.reload(importlib.import_module('db.migrations'))
from db.migrations import create_transactions,upload_transactions

master_transactions, slaves_transactions = create_transactions(transactions)
upload_transactions(get_db, master_transactions, slaves_transactions)

100%|██████████| 15/15 [00:02<00:00,  7.03it/s]


### processus pour faire un id qui marche

In [49]:
import importlib
import pandas as pd
importlib.reload(importlib.import_module('api.bank_api'))
from api.bank_api import connect_to_bank

connect_to_bank("LCL_CRLYFRPP",requisition_id='645d79b0-c29d-47b0-9fdd-c83b9b15e807')

2025-10-16 23:05:54.053 | INFO     | api.bank_api:connect_to_bank:35 - Requisition ID:


{'id': '645d79b0-c29d-47b0-9fdd-c83b9b15e807',
 'created': '2025-10-16T20:39:51.302602Z',
 'redirect': 'https://gocardless.com',
 'status': 'LN',
 'institution_id': 'LCL_CRLYFRPP',
 'agreement': 'e8d4935c-9453-41f2-b644-702b3cd7a179',
 'reference': '46341f98-572d-45d2-9120-e092a9af64b8',
 'accounts': ['1bdb6174-5522-44d5-a67e-28d50123966f'],
 'link': 'https://ob.gocardless.com/ob-psd2/start/0507a7ca-4f8e-47ac-b1b9-1495a1eb4750/LCL_CRLYFRPP',
 'ssn': None,
 'account_selection': False,
 'redirect_immediate': False}

In [67]:
client.account_api(id="1bdb6174-5522-44d5-a67e-28d50123966f").get_balances()

{'balances': [{'balanceAmount': {'amount': '-446.98', 'currency': 'EUR'},
   'balanceType': 'expected',
   'referenceDate': '2025-10-16'},
  {'balanceAmount': {'amount': '1543.02', 'currency': 'EUR'},
   'balanceType': 'closingBooked',
   'referenceDate': '2025-10-15'}]}

In [84]:
account_id,current_amount=df_accounts[df_accounts['name'].str.contains("LCL")][['account_id','current_amount']].iloc[0]

In [85]:
account_id

'5e139075-c4b5-4f6f-afdb-e8a5efa9d377'

In [93]:
from utils.secrets import save_secret
from db.models import AccountsSecretsCreate
from datetime import datetime
account=AccountsSecretsCreate(  
    accountId=account_id,
    bankId="LCL_CRLYFRPP",
    updated_at=datetime.now().isoformat(),
    secretId="1bdb6174-5522-44d5-a67e-28d50123966f")
save_secret(account)

In [10]:
api.get_metadata()

HTTPError: {'response': {'summary': 'Authentication failed', 'detail': 'Authentication credentials were not provided.', 'status_code': 401}, 'status': 401}

In [ ]:
accounts["accounts"] # les id des différents comptes

['a52da9b8-2983-4428-bfe9-e1e2c85dbcb7',
 'f47d8c6f-6bca-41ae-9469-b4f88ecb2e61']